In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime

In [24]:
df = pd.read_csv("../data/rides_combined.csv", index_col=0)
df.head(2)

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,...,cancellation_reason,cancellation_comment,sheet_name,file_name,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two
0,5727475e-8224-4302-9228-c92b9d4a5220,f8ff0526-887a-4e48-ad96-977e12fd70c1,5483,1.0,4.65,4.65,0.0,0.0,STANDARD,Würzmühle,...,NaN,NaN,1,b'Rides_2021-07.xlsx',NaN,NaN,NaN,NaN,NaN,NaN
1,18fec0a6-b7ba-442b-8472-04bdb6ba1b86,51e1a1a8-995c-488c-84ce-3789e46f0417,3575,1.0,0.00,2.77,0.0,0.0,BAHN_CARD,Julius-Wilde-Straße,...,NaN,NaN,1,b'Rides_2021-07.xlsx',NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_stops = pd.read_excel("../data/MoDstops+Preismodell.xlsx", sheet_name="MoDstops")
df_stops.head(2)

,MoDStop Id,MoDStop Name,MoDStop Lat,MoDStop Long,MoDStop Adresse
0,1001,Mandelgasse,49.351780,8.129000,"Seilerbahn 1, 67433 Neustadt"
1,1002,Hauptfeuerwache,49.353733,8.131552,"Lindenstraße 11, 67433 Neustadt"


#### Column: id

In [13]:
print("# of NaN:", int(df[['id']].isna().sum()))
print("# of duplicates:", df.id.duplicated().sum() - int(df[['id']].isna().sum() - 1), "(exklusive NaN)")

# of NaN: 4535
# of duplicates: 4 (exklusive NaN)


In [10]:
df.loc[(df['id'].duplicated()) & (~df['id'].isna()), :]

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,...,cancellation_reason,cancellation_comment,sheet_name,file_name,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two
1607,b2246a46-183f-4966-8d60-5120985239c9,227d25b5-9f23-4c15-8b96-ec9541a0fd87,1873,1.0,2.85,2.85,2.85,0.0,STANDARD,Schütt,...,NaN,NaN,5,b'Rides_2021-09.xlsx',NaN,NaN,NaN,NaN,NaN,NaN
2303,30220736-f869-4e77-9035-12a675ddfeb6,d253df67-6600-449e-add7-ad95c84fda85,4505,1.0,0.00,4.00,0.00,0.0,BAHN_CARD,Am Kirschgarten,...,NaN,NaN,27,b'Rides_2021-09.xlsx',NaN,NaN,NaN,NaN,NaN,NaN
2565,b254fd87-6e8a-4b7c-96e6-a4af48e2ec22,be6038b5-1582-4efb-9c80-8d07c1749e7d,7581,4.0,16.50,16.50,16.50,0.0,STANDARD,Am Kirschgarten,...,NaN,NaN,2,b'Rides_2021-10.xlsx',NaN,NaN,NaN,NaN,NaN,NaN
3119,bb4c0149-4c38-4018-a096-d90ef14ee801,456d1e59-257e-42b7-8e3d-94ddd24f88d5,6392,1.0,5.10,5.10,5.10,0.0,STANDARD,Altes Schulhaus Lachen,...,NaN,NaN,20,b'Rides_2021-10.xlsx',NaN,NaN,NaN,NaN,NaN,NaN


Wieso gibts doppelte ids?

Fehlende ID's sind problematisch, wenn wir die ride id als Primärschlüssel nutzen wollen. Es handelt sich um rides, bei denen der Status 'offer' oder 'offer_rejected' ist. 

#### Column: user_id

In [79]:
print("# of NaN:", int(df[['user_id']].isna().sum()))
print("# of duplicates:", df.user_id.duplicated().sum() - int(df[['user_id']].isna().sum()), "(exklusive NaN)")

# of NaN: 0
# of duplicates: 10458 (exklusive NaN)


#### Column: distance

In [59]:
print("# of NaN:", int(df[['distance']].isna().sum()))
print("# of neg:", len(df.loc[(df.distance < 0)]))
print("# of zero:", len(df.loc[(df.distance == 0)]))
print("# of ones:", len(df.loc[(df.distance == 1)]))

# of NaN: 0
# of neg: 0
# of zero: 0
# of ones: 24


In [60]:
comparison = np.where(df['pickup_address'] == df['dropoff_address'], True, False)
df.loc[:, 'isSameAddress'] = comparison
mask = df['distance']==1
df[mask][['pickup_address', 'dropoff_address', 'isSameAddress']]

,pickup_address,dropoff_address,isSameAddress
37,Schütt,Schütt,True
38,Schütt,Schütt,True
400,49.330245|8.130933,49.330245|8.130933,True
430,49.351731|8.136637,49.351731|8.136637,True
448,49.316193|8.135477,49.316193|8.135477,True
822,49.376347|8.153444,49.376347|8.153444,True
841,49.376347|8.153444,49.376347|8.153444,True
845,Le Quartier Hornbach,Le Quartier Hornbach,True
866,49.35552|8.136674,49.35552|8.136674,True
881,Strohmarkt,Strohmarkt,True


Was sollen wir mit distance=1m und gleichem Start- & Endhaltestopp machen? 

#### Column: number_of_passenger

In [14]:
print("# of NaN:", int(df[['number_of_passenger']].isna().sum()))
print("# of neg:", len(df.loc[(df.number_of_passenger < 0)]))
print("# of zero:", len(df.loc[(df.number_of_passenger == 0)]))
print("# of pos:", len(df.loc[(df.number_of_passenger > 0)]))
print('Max # of passengers:', df['number_of_passenger'].max())

# of NaN: 4535
# of neg: 0
# of zero: 0
# of pos: 5817
Max # of passengers: 5.0


Fehlende Angaben bei number_of_passenger: Es handelt sich um rides, bei denen der Status 'offer' oder 'offer_rejected' ist. 
--> Bei Analysen vermutlich sowieso filtern auf completed

#### Column: price_operations

In [15]:
print("# of NaN:", int(df[['price_operations']].isna().sum()))
print("# of neg:", len(df.loc[(df.price_operations < 0)]))
print("# of zero:", len(df.loc[(df.price_operations == 0)]))
print("# of pos:", len(df.loc[(df.price_operations > 0)]))
print('Min price:', df['price_operations'].min())
print('Max price:', df['price_operations'].max())

# of NaN: 1204
# of neg: 0
# of zero: 459
# of pos: 8689
Min price: 0.0
Max price: 33.1


#### Column: price_offer

In [17]:
print("# of NaN:", int(df[['price_offer']].isna().sum()))
print("# of neg:", len(df.loc[(df.price_offer < 0)]))
print("# of zero:", len(df.loc[(df.price_offer == 0)]))
print("# of pos:", len(df.loc[(df.price_offer > 0)]))
print('Min price:', df['price_offer'].min())
print('Max price:', df['price_offer'].max())

# of NaN: 4535
# of neg: 0
# of zero: 0
# of pos: 5817
Min price: 0.8
Max price: 32.3


#### Column: price_payed

In [18]:
print("# of NaN:", int(df[['price_payed']].isna().sum()))
print("# of neg:", len(df.loc[(df.price_payed < 0)]))
print("# of zero:", len(df.loc[(df.price_payed == 0)]))
print("# of pos:", len(df.loc[(df.price_payed > 0)]))
print('Min price:', df['price_payed'].min())
print('Max price:', df['price_payed'].max())

# of NaN: 4535
# of neg: 0
# of zero: 1323
# of pos: 4494
Min price: 0.0
Max price: 555.33


#### Column: free_ride

In [19]:
print("# of NaN:", int(df[['free_ride']].isna().sum()))


# of NaN: 4535


In [22]:
df.free_ride.value_counts()

0.0    5174
1.0     643
Name: free_ride, dtype: int64

#### Column: pickup_address and dropoff_address

In [23]:
print("# of NaN:", int(df[['pickup_address']].isna().sum()))
print("# of NaN:", int(df[['dropoff_address']].isna().sum()))


# of NaN: 0
# of NaN: 0


In [45]:
df['start_id'] = ""
df['end_id'] = ""

def get_stop_id(address):
    if address[0].isdigit():
        lat = address.split("|")[0]
        long = address.split("|")[1]
        for index, row in df_stops.iterrows():
            if str(row["MoDStop Lat"]) == lat and str(row["MoDStop Long"]) == long:
                return row["MoDStop Id"]
        return "No match of lat and long"
    else:
        # different naming
        if address == "Rewe Mußbach":
            address = address + " (Shoppenwiese)"
        for index, row in df_stops.iterrows():
            if row["MoDStop Name"] == address:
                return row["MoDStop Id"]
        return "No match of address name"
            		
        

In [39]:
for index, row in df.iterrows():
    df.at[index, 'start_id'] = get_stop_id(row['pickup_address'])
    df.at[index, 'end_id'] = get_stop_id(row['dropoff_address'])

In [ ]:
df.to_excel("test.xlsx")
